## adi_cmac2 dask scaling

In [1]:
import subprocess
import os
import sys
from dask_jobqueue import PBSCluster
from distributed import Client, progress
from datetime import datetime, timedelta
from pkg_resources import load_entry_point

In [2]:
dir(adi_cmac2)

NameError: name 'adi_cmac2' is not defined

In [3]:
def exec_adi(info_dict):
    facility = info_dict['facility']
    site = info_dict['site']
    start_date = info_dict['start_date']
    end_date = info_dict['end_date']
    logs_dir = "/home/rjackson/adi_logs"
    os.environ["DATASTREAM_DATA"] = "/lustre/or-hydra/cades-arm/rjackson/"
    logs_dir += logs_dir + "/" + site + start_date + "_" + end_date
    if not os.path.isdir(logs_dir):
        os.makedirs(logs_dir)
    os.environ["LOGS_DATA"] = logs_dir
    os.environ["PROJ_LIB"] = "/home/rjackson/anaconda3/envs/adi_env3/share/proj/"
    print(("/home/rjackson/anaconda3/envs/adi_env3/bin/adi_cmac2 -D 1 -f " + 
                     facility + " -s " + site + " -b " + start_date + " -e "+  end_date))
    subprocess.call(("/home/rjackson/anaconda3/envs/adi_env3/bin/adi_cmac2 -D 1 -f " + 
                     facility + " -s " + site + " -b " + start_date + " -e "+  end_date), shell=True)
    #sys.argv[0] = "/home/rjackson/anaconda3/envs/adi_env3/bin/adi_cmac2"
    #sys.argv[1] = "-D"
    #sys.argv[2] = "1"
    #sys.argv[3] = "-f"
    #sys.argv[4] = facility
    #sys.argv[5] = "-s"
    #sys.argv[6] = start_date
    #sys.argv[7] = "-e"
    #sys.argv[8] = end_date
    #return load_entry_point('vap-adi-cmac2==1.0.0.dev0', 'console_scripts', 'adi_cmac2')   

In [4]:
the_cluster = PBSCluster(processes=6, cores=36, queue="arm_high_mem", 
                         walltime="3:00:00", resource_spec="qos=std",
                         job_extra=["-A arm", "-W group_list=cades-arm"],
                         env_extra=[". /home/rjackson/anaconda3/etc/profile.d/conda.sh", "conda activate adi_env3"])
the_cluster.scale(36)

In [5]:
client = Client(the_cluster)
client

Client Scheduler: tcp://10.23.216.82:44276 Dashboard: http://10.23.216.82:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [9]:
client

Client Scheduler: tcp://10.23.216.82:44276 Dashboard: http://10.23.216.82:8787/status,Cluster Workers: 36 Cores: 216 Memory: 1.62 TB


In [21]:
the_func = lambda x, y: exec_adi("i5", "sgp")


In [7]:
def make_date_list_dict_list(start_day, end_day):
    cur_day = start_day
    the_list = []
    while(cur_day < end_day):
        next_day = cur_day + timedelta(days=1)
        temp_dict = {}
        temp_dict['facility'] = "I5"
        temp_dict['site'] = "sgp"
        temp_dict['start_date'] = cur_day.strftime("%Y%m%d")
        temp_dict['end_date'] = next_day.strftime("%Y%m%d")
        the_list.append(temp_dict)
        cur_day = cur_day + timedelta(days=1)
    return the_list
date_list = make_date_list_dict_list(datetime(2019, 1, 1), datetime(2019,2,6))
print(date_list)

[{'facility': 'I5', 'site': 'sgp', 'start_date': '20190101', 'end_date': '20190102'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190102', 'end_date': '20190103'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190103', 'end_date': '20190104'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190104', 'end_date': '20190105'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190105', 'end_date': '20190106'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190106', 'end_date': '20190107'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190107', 'end_date': '20190108'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190108', 'end_date': '20190109'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190109', 'end_date': '20190110'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190110', 'end_date': '20190111'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190111', 'end_date': '20190112'}, {'facility': 'I5', 'site': 'sgp', 'start_date': '20190112', 'end

In [10]:
futures = client.map(exec_adi, date_list)

In [16]:
futures

[<Future: status: finished, type: NoneType, key: exec_adi-99403568728d730057c7dfd6a9f3975b>,
 <Future: status: finished, type: NoneType, key: exec_adi-1f0256d6c03ff7027348c97929f8b431>,
 <Future: status: pending, key: exec_adi-4d2cae1f2b675dba3848b5708a4bece6>,
 <Future: status: pending, key: exec_adi-e258cd856100870c39c4b21e3d05c8dd>,
 <Future: status: pending, key: exec_adi-3edacab31cf69d297f316193203d6f7d>,
 <Future: status: pending, key: exec_adi-8e330e34c0a7c77e08a9630eed3ddecc>,
 <Future: status: pending, key: exec_adi-edb716d3bf4d23f653425d5663b19f54>,
 <Future: status: pending, key: exec_adi-9ae4135ced91fc86eb81d09cca9eb478>,
 <Future: status: pending, key: exec_adi-ab9228f82e5dbf4dc3e3543dda1c26a6>,
 <Future: status: pending, key: exec_adi-e29f7ec520d40127f4f21479e5c2bb3d>,
 <Future: status: pending, key: exec_adi-4733dab740194b866a03847e2f3a4b93>,
 <Future: status: pending, key: exec_adi-5f3da22bfbba6a546e986c4a7e5982ed>,
 <Future: status: pending, key: exec_adi-c83260a7dcbae

In [94]:
del futures

In [60]:
the_cluster.job_script()

'#!/bin/bash\n\n#!/usr/bin/env bash\n#PBS -N dask-worker\n#PBS -q arm_high_mem\n#PBS -A arm\n#PBS -l qos=std\n#PBS -l walltime=3:00:00\n#PBS -A arm\n#PBS -W group_list=cades-arm\nJOB_ID=${PBS_JOBID%.*}\n\nconda activate adi_env3\n\n/home/rjackson/anaconda3/envs/adi_env3/bin/python -m distributed.cli.dask_worker tcp://10.23.216.82:45007 --nthreads 6 --nprocs 6 --memory-limit 45.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --local-directory $localscratch --interface ib0 --interface ib0 --interface ib0\n'

In [120]:
exec_adi(date_list[0])

/home/rjackson/anaconda3/envs/adi_env3/bin/adi_cmac2 -D 1 -f I5 -s sgp -b 20190101 -e 20190102


KeyboardInterrupt: 

In [ ]:
os.path.i